In [1]:
import os 
from dotenv import load_dotenv

# load_dotenv("../../.env")

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"]="ROLE_BASED_ACCESS_CONTROL"
os.environ["LANGSMITH_ENDPOINT"]=os.getenv("LANGSMITH_ENDPOINT")

In [2]:
!echo $LANGCHAIN_PROJECT

ROLE_BASED_ACCESS_CONTROL


In [3]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_core.messages import HumanMessage,SystemMessage,ToolMessage
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_chroma import Chroma 
from langchain.chains.combine_documents import create_stuff_documents_chain  
from langchain_openai import ChatOpenAI


#### Calling LLM

In [4]:
llm=ChatOpenAI(model="gpt-5")

#### Embedding model 

In [5]:
embedding=OpenAIEmbeddings(model="text-embedding-3-large")

/var/folders/dd/ffn14k7x4bz7kmmd0l7kmqg80000gn/T/ipykernel_10945/2347312799.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding=OpenAIEmbeddings(model="text-embedding-3-large")


#### Loading vector DB 

In [6]:
db2=Chroma(persist_directory="../vector_data/Engineering_doc",embedding_function=embedding)

#### Converted vector db into retriever 

In [7]:
retriever=db2.as_retriever()

In [8]:
output=retriever.invoke("Company Overview")

In [9]:
retriever.get_relevant_documents("Company Overview ")

/var/folders/dd/ffn14k7x4bz7kmmd0l7kmqg80000gn/T/ipykernel_10945/3935937752.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("Company Overview ")


[Document(id='0f3aa999-943b-459a-ba0d-4360729afa02', metadata={'Header 1': 'FinSolve Technologies Engineering Document', 'Header 2': '1. Introduction', 'Header 3': '1.1 Company Overview'}, page_content='# FinSolve Technologies Engineering Document  \n## 1. Introduction  \n### 1.1 Company Overview\nFinSolve Technologies is a leading FinTech company headquartered in Bangalore, India, with operations across North America, Europe, and Asia-Pacific. Founded in 2018, FinSolve provides innovative financial solutions, including digital banking, payment processing, wealth management, and enterprise financial analytics, serving over 2 million individual users and 10,000 businesses globally.'),
 Document(id='b3e5b5ea-ee0d-414e-a50f-1d7584e86ec1', metadata={'Header 1': 'FinSolve Technologies Engineering Document', 'Header 3': '9.2 Long-Term Strategic Direction (2026–2027)', 'Header 2': '9. Future Roadmap'}, page_content='### 9.2 Long-Term Strategic Direction (2026–2027)  \n#### 9.2.1 Global Market

In [10]:
prompt=ChatPromptTemplate.from_template(
    """
    You are a helpful assistant. Your name is FinSolve bot.
    - output should be like first line user question must be there
    - answer question of user using below context 
    <context>
    {context}
    </context>
    <user_input>
    {input}
    </user_input>
    """
)



In [11]:
chain=create_stuff_documents_chain(llm,prompt)

In [12]:
user="hi,i am abhijit and tell me about company"

result=chain.invoke({"input":user,
                     "context":retriever.get_relevant_documents(user)
})

In [13]:
print(result)

hi,i am abhijit and tell me about company

Hi Abhijit! Here’s a quick overview of FinSolve Technologies:

- Who we are: A leading FinTech company founded in 2018, headquartered in Bangalore, India, with operations across North America, Europe, and Asia-Pacific.
- What we do: Digital banking, payment processing, wealth management, and enterprise financial analytics.
- Who we serve: 2+ million individual users and 10,000 businesses globally.

What’s coming next (Q2–Q4 2025):
- AI/ML:
  - Personalized financial insights (spending pattern recognition, fraud anomaly detection, budget recommendations)
  - Chatbot/virtual financial assistant with multi-language support
- Blockchain/Crypto:
  - Crypto wallet integration; support for Bitcoin and Ethereum
  - Blockchain-based transaction verification
  - Smart contracts for automated lending and escrow with transparent audit trails
- Localization:
  - New languages: Hindi, Spanish, Mandarin, Arabic; RTL support
  - Regional compliance adapters a